In [63]:
#import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import tree, preprocessing, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import itertools

In [64]:
#penguins data
url = "https://philchodrow.github.io/PIC16A/datasets/palmer_penguins.csv"
penguins = pd.read_csv(url)
penguins.head()

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11/11/07,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11/11/07,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN
3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,11/16/07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adult not sampled.
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,11/16/07,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,NaN


In [65]:
#dropping columns: studyName, Individual ID, Date Egg, Sample Number
pdata = penguins.drop(['studyName', 'Individual ID', 'Date Egg', 'Region', 'Sample Number', 'Comments'], axis = 1)
pdata.head()

,Species,Island,Stage,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,Adelie Penguin (Pygoscelis adeliae),Torgersen,"Adult, 1 Egg Stage",Yes,39.1,18.7,181.0,3750.0,MALE,NaN,NaN
1,Adelie Penguin (Pygoscelis adeliae),Torgersen,"Adult, 1 Egg Stage",Yes,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,"Adult, 1 Egg Stage",Yes,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,"Adult, 1 Egg Stage",Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelie Penguin (Pygoscelis adeliae),Torgersen,"Adult, 1 Egg Stage",Yes,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426


In [66]:
#checking info 
pdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Species              344 non-null    object 
 1   Island               344 non-null    object 
 2   Stage                344 non-null    object 
 3   Clutch Completion    344 non-null    object 
 4   Culmen Length (mm)   342 non-null    float64
 5   Culmen Depth (mm)    342 non-null    float64
 6   Flipper Length (mm)  342 non-null    float64
 7   Body Mass (g)        342 non-null    float64
 8   Sex                  334 non-null    object 
 9   Delta 15 N (o/oo)    330 non-null    float64
 10  Delta 13 C (o/oo)    331 non-null    float64
dtypes: float64(6), object(5)
memory usage: 29.7+ KB


In [67]:
#drop nan values
mydata = pdata.dropna()
mydata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 1 to 343
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Species              325 non-null    object 
 1   Island               325 non-null    object 
 2   Stage                325 non-null    object 
 3   Clutch Completion    325 non-null    object 
 4   Culmen Length (mm)   325 non-null    float64
 5   Culmen Depth (mm)    325 non-null    float64
 6   Flipper Length (mm)  325 non-null    float64
 7   Body Mass (g)        325 non-null    float64
 8   Sex                  325 non-null    object 
 9   Delta 15 N (o/oo)    325 non-null    float64
 10  Delta 13 C (o/oo)    325 non-null    float64
dtypes: float64(6), object(5)
memory usage: 30.5+ KB


In [68]:
#get rid of penguin with period for sex
data = mydata.drop(mydata.index[318])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 1 to 343
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Species              324 non-null    object 
 1   Island               324 non-null    object 
 2   Stage                324 non-null    object 
 3   Clutch Completion    324 non-null    object 
 4   Culmen Length (mm)   324 non-null    float64
 5   Culmen Depth (mm)    324 non-null    float64
 6   Flipper Length (mm)  324 non-null    float64
 7   Body Mass (g)        324 non-null    float64
 8   Sex                  324 non-null    object 
 9   Delta 15 N (o/oo)    324 non-null    float64
 10  Delta 13 C (o/oo)    324 non-null    float64
dtypes: float64(6), object(5)
memory usage: 30.4+ KB


In [69]:
#transforming cols that are not floats and ints
le = preprocessing.LabelEncoder()

#species 
data['Species'] = le.fit_transform(data['Species'])
#island
data['Island'] = le.fit_transform(data['Island'])
#stage
data['Stage'] = le.fit_transform(data['Stage'])
#clutch competition
data['Clutch Completion'] = le.fit_transform(data['Clutch Completion'])
#sex
data['Sex'] = le.fit_transform(data['Sex'])

data.head()

,Species,Island,Stage,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
1,0,2,0,1,39.5,17.4,186.0,3800.0,0,8.94956,-24.69454
2,0,2,0,1,40.3,18.0,195.0,3250.0,0,8.36821,-25.33302
4,0,2,0,1,36.7,19.3,193.0,3450.0,0,8.76651,-25.32426
5,0,2,0,1,39.3,20.6,190.0,3650.0,1,8.66496,-25.29805
6,0,2,0,0,38.9,17.8,181.0,3625.0,0,9.18718,-25.21799


In [70]:
#making combos
categories = list(data.columns)[1:]
combos = list(itertools.combinations(categories, 3))
combos

[('Island', 'Stage', 'Clutch Completion'),
 ('Island', 'Stage', 'Culmen Length (mm)'),
 ('Island', 'Stage', 'Culmen Depth (mm)'),
 ('Island', 'Stage', 'Flipper Length (mm)'),
 ('Island', 'Stage', 'Body Mass (g)'),
 ('Island', 'Stage', 'Sex'),
 ('Island', 'Stage', 'Delta 15 N (o/oo)'),
 ('Island', 'Stage', 'Delta 13 C (o/oo)'),
 ('Island', 'Clutch Completion', 'Culmen Length (mm)'),
 ('Island', 'Clutch Completion', 'Culmen Depth (mm)'),
 ('Island', 'Clutch Completion', 'Flipper Length (mm)'),
 ('Island', 'Clutch Completion', 'Body Mass (g)'),
 ('Island', 'Clutch Completion', 'Sex'),
 ('Island', 'Clutch Completion', 'Delta 15 N (o/oo)'),
 ('Island', 'Clutch Completion', 'Delta 13 C (o/oo)'),
 ('Island', 'Culmen Length (mm)', 'Culmen Depth (mm)'),
 ('Island', 'Culmen Length (mm)', 'Flipper Length (mm)'),
 ('Island', 'Culmen Length (mm)', 'Body Mass (g)'),
 ('Island', 'Culmen Length (mm)', 'Sex'),
 ('Island', 'Culmen Length (mm)', 'Delta 15 N (o/oo)'),
 ('Island', 'Culmen Length (mm)', 'De

In [71]:
#splitting the data in test and training
train, test = train_test_split(data, test_size = 0.3)
train.shape, test.shape

((226, 11), (98, 11))

In [72]:
x_train = train.drop(['Species'], axis = 1)
y_train = train['Species'].values

x_test = test.drop(['Species'], axis = 1)
y_test = test['Species'].values

In [73]:
x_train

,Island,Stage,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
305,0,0,1,50.8,17.3,228.0,5600.0,1,8.27428,-26.30019
278,0,0,1,43.2,14.5,208.0,4450.0,0,8.48367,-26.86485
164,1,0,1,47.0,17.3,185.0,3700.0,0,8.72037,-24.80526
270,0,0,1,46.6,14.2,210.0,4850.0,0,8.38289,-26.86352
267,0,0,0,50.5,15.9,225.0,5400.0,1,8.65803,-26.57585
...,...,...,...,...,...,...,...,...,...,...
269,0,0,1,45.2,15.8,215.0,5300.0,1,8.55868,-26.67799
117,2,0,1,37.3,20.5,199.0,3775.0,1,9.49645,-26.36678
183,1,0,1,54.2,20.8,201.0,4300.0,1,9.49283,-24.59996
115,0,0,1,42.7,18.3,196.0,4075.0,1,8.91434,-26.42018


In [74]:
def check_column_score(cols):
    """
    Trains and evaluates a model via cross-validation on the columns of the data
    with selected indices
    """
    clf = LogisticRegressionCV(cv=5, random_state=0, max_iter = 1500).fit(x_train[cols], y_train)
    return clf.score(x_train[cols], y_train)
    

clf = LogisticRegressionCV(cv=5, random_state=0).fit(X, y)

logreg = LogisticRegression(max_iter = 1000)
return cross_val_score(logreg, x_train[cols], y_train, cv = 5).mean()

clf.score(X, y)

In [75]:
D = {}

for i in range(len(combos)): 
    cols = list(combos[i])
    x = check_column_score(cols)
    D[i] = x
    

In [76]:
for i in range(10):
    print(str(i) +" : " + str(D[i]))

0 : 0.6814159292035398
1 : 0.9513274336283186
2 : 0.7964601769911505
3 : 0.8495575221238938
4 : 0.7831858407079646
5 : 0.6814159292035398
6 : 0.8230088495575221
7 : 0.827433628318584
8 : 0.9646017699115044
9 : 0.7831858407079646


In [77]:
def test_column_score(tupCol):
    """
    Trains and evaluates a model on the test set using the columns of the data
    with selected indices
    """
    cols = list(tupCol)
    logreg = LogisticRegression(max_iter = 1000)
    logreg.fit(x_train[cols], y_train)
    return logreg.score(x_test[cols], y_test)

In [78]:
L = list(D.items())
L.sort(key = lambda tup: tup[1], reverse = True)

best = L[0:10]

for where, score in best:
    print("---- " + str(where))
    print(str(combos[where]) + ", Score: " + str(np.round(score, 5)))
    test_score = test_column_score(combos[where])
    print("Test score is: " + str(np.round(test_score, 5)))

---- 85
('Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)'), Score: 1.0
Test score is: 0.95918
---- 89
('Culmen Length (mm)', 'Culmen Depth (mm)', 'Delta 13 C (o/oo)'), Score: 1.0
Test score is: 1.0
---- 94
('Culmen Length (mm)', 'Body Mass (g)', 'Sex'), Score: 1.0
Test score is: 0.96939
---- 15
('Island', 'Culmen Length (mm)', 'Culmen Depth (mm)'), Score: 0.99558
Test score is: 0.95918
---- 86
('Culmen Length (mm)', 'Culmen Depth (mm)', 'Body Mass (g)'), Score: 0.99115
Test score is: 0.9898
---- 87
('Culmen Length (mm)', 'Culmen Depth (mm)', 'Sex'), Score: 0.99115
Test score is: 0.9898
---- 88
('Culmen Length (mm)', 'Culmen Depth (mm)', 'Delta 15 N (o/oo)'), Score: 0.98673
Test score is: 0.9898
---- 90
('Culmen Length (mm)', 'Flipper Length (mm)', 'Body Mass (g)'), Score: 0.98673
Test score is: 0.95918
---- 16
('Island', 'Culmen Length (mm)', 'Flipper Length (mm)'), Score: 0.9823
Test score is: 0.94898
---- 91
('Culmen Length (mm)', 'Flipper Length (mm)', 'Sex'), Score:

In [ ]:
#where i stopped understadning what was going on











In [53]:
X_train = train[list(combos[0])]
X_test = test[list(combos[0])]

In [60]:
X_train

,Island,Stage,Clutch Completion
64,0,0,1
50,0,0,1
338,0,0,0
174,1,0,0
322,0,0,1
...,...,...,...
288,0,0,1
198,1,0,1
116,2,0,1
279,0,0,1


In [80]:
#logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))

In [86]:
lr = LogisticRegression(max_iter = 1000)
lr.fit(x_train, y_train)

lr.score(x_test, y_test)
#logit_roc_auc = roc_auc_score(y_test, lr.predict(x_test), multi_class = 'ovo')



1.0

In [74]:
x_test

,Island,Stage,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
138,1,0,0,37.0,16.5,185.0,3400.0,0,8.61651,-26.07021
230,0,0,0,40.9,13.7,214.0,4650.0,0,8.19579,-25.39330
177,1,0,0,52.0,19.0,197.0,4150.0,1,9.36799,-24.47142
264,0,0,1,50.5,15.9,222.0,5550.0,1,8.46894,-26.60436
201,1,0,1,49.8,17.3,198.0,3675.0,0,9.32169,-24.41562
...,...,...,...,...,...,...,...,...,...,...
4,2,0,1,36.7,19.3,193.0,3450.0,0,8.76651,-25.32426
196,1,0,1,50.9,17.9,196.0,3675.0,0,9.43684,-24.16566
197,1,0,1,50.8,18.5,201.0,4450.0,1,9.45827,-24.35575
157,1,0,1,45.2,17.8,198.0,3950.0,0,8.88942,-24.49433


In [76]:
y_test

array([0, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 0, 2, 0, 2, 1, 2, 0, 1, 0, 0, 1,
       2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 1, 0, 2, 0, 0, 0, 0,
       0, 0, 1, 2, 0, 1, 2, 1, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 1, 0,
       2, 1, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 2, 1, 2, 1, 0, 1, 0, 2, 2, 0,
       2, 2, 0, 2, 0, 0, 1, 1, 1, 2])

In [83]:
aaa = lr.predict(x_test)
aaa

array([0, 0, 1, 1, 2, 2, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0,
       0, 0, 0, 1, 2, 1, 1, 2, 2, 1, 1, 2, 0, 1, 2, 2, 1, 0, 1, 1, 0, 0,
       2, 2, 0, 2, 0, 2, 0, 1, 0, 0, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 1, 0,
       1, 2, 1, 1, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 2, 0, 0, 1, 2, 0, 2,
       0, 2, 0, 0, 0, 1, 0, 2, 1, 1])

In [79]:
yy_test = data[['Species']]

In [80]:
yy_test

,Species
1,0
2,0
4,0
5,0
6,0
...,...
338,2
340,2
341,2
342,2


In [1]:
x_test

NameError: name 'x_test' is not defined

In [ ]:
aaa